In [13]:
%pip install -q torch
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import torch
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-1.7B")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

C:\tools\Manim\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (r

In [ ]:
import time
import threading
from queue import Queue

def generate_with_timing(model, tokenizer, use_cache, prompt, device, results_queue):
    tokens = tokenizer.encode(prompt, return_tensors="pt").to(device)
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)
    
    # Armazena o texto gerado
    generated_text = []
    
    # Inicia o timer
    start_time = time.time()
    
    # Thread de geração
    thread = threading.Thread(
        target=model.generate,
        kwargs={
            "input_ids": tokens,
            "max_new_tokens": 100,
            "use_cache": use_cache,
            "streamer": streamer
        }
    )
    thread.start()
    
    # Consome o streamer
    for text in streamer:
        generated_text.append(text)
    
    thread.join()
    end_time = time.time()
    
    # Calcula métricas
    elapsed_time = end_time - start_time
    full_text = "".join(generated_text)
    tokens_generated = len(tokenizer.encode(full_text)) - len(tokens[0])
    tokens_per_second = tokens_generated / elapsed_time if elapsed_time > 0 else 0
    
    # Armazena resultados
    results_queue.put({
        "use_cache": use_cache,
        "time": elapsed_time,
        "tokens_generated": tokens_generated,
        "tokens_per_second": tokens_per_second,
        "text": full_text
    })

# Execução do benchmark
def run_benchmark(model, tokenizer, prompt, device):
    """
    Executa o benchmark comparando use_cache=True vs False
    """
    results_queue = Queue()
    threads = []
    
    print("Iniciando benchmark...\n")
    
    # Cria e inicia as threads
    for use_cache in [True, False]:
        thread = threading.Thread(
            target=generate_with_timing,
            args=(model, tokenizer, use_cache, prompt, device, results_queue)
        )
        threads.append(thread)
        thread.start()
    
    # Aguarda todas as threads terminarem
    for thread in threads:
        thread.join()
    
    # Coleta resultados
    results = []
    while not results_queue.empty():
        results.append(results_queue.get())
    
    # Ordena por use_cache para facilitar comparação
    results.sort(key=lambda x: x["use_cache"], reverse=True)
    
    print("=" * 70)
    print("RESULTADOS")
    print("=" * 70)
    
    for result in results:
        print(f"\n{'COM CACHE' if result['use_cache'] else 'SEM CACHE'}")
        print(f"  Tempo total: {result['time']:.2f}s")
        print(f"  Tokens gerados: {result['tokens_generated']}")
        print(f"  Velocidade: {result['tokens_per_second']:.2f} tokens/s")
        print(f"  Texto: {result['text'][:100]}...")
    
    # Comparação
    if len(results) == 2:
        speedup = results[0]['time'] / results[1]['time'] if results[1]['time'] > 0 else 0
        print(f"SPEEDUP: {speedup:.2f}x ")

In [5]:
prompt = "Black Panther was a"
run_benchmark(model, tokenizer, prompt, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


🚀 Iniciando benchmark...

📊 RESULTADOS DO BENCHMARK

🔥 COM CACHE
  Tempo total: 52.11s
  Tokens gerados: 100
  Velocidade: 1.92 tokens/s
  Texto: Black Panther was a great movie. I was really excited to see it. I was really excited to see it. I w...

❄️  SEM CACHE
  Tempo total: 97.25s
  Tokens gerados: 100
  Velocidade: 1.03 tokens/s
  Texto: Black Panther was a great movie. I was really excited to see it. I was really excited to see it. I w...

⚡ SPEEDUP: 0.54x (COM cache é mais rápido)

